In [1]:
import pandas as pd
import json
from pprint import pformat
import sqlite3
#
from functools import wraps
import time


def timeit(func):
    @wraps(func)
    def timeit_wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        total_time = end_time - start_time
        print(f'Function {func.__name__} Took {total_time:.4f} seconds')
        return result
    return timeit_wrapper

In [36]:
@timeit
def get_set_from_json(file_name:str):
    with open(file_name) as f:
        data = json.load(f)

    lotr_set = data['data']['cards']

    return lotr_set

lotr_set = get_set_from_json('ltr.json')

Function get_set_from_json Took 0.0233 seconds


In [32]:
drop_list = ['artist','availability','borderColor','colors','colorIdentity',
    'convertedManaCost','edhrecRank','colorIdentity','foreignData',
    'frameVersion','isStarter','language','layout','legalities',
    'manaCost','manaValue','printings','securityStamp','setCode',
    'power', 'toughness', 'edhrecSaltiness','isReprint','flavorName',
    'hasAlternativeDeckLimit']

all_keys = set().union(*(d.keys() for d in lotr_set))
all_keys = sorted(all_keys)
new_dict = {}
lists_keys = []
dicts_keys = []
for key in all_keys:
    new_dict[key] = [card[key] if key in card else None for card in lotr_set]
for key in drop_list:
    if key in new_dict:
        new_dict.pop(key)
for key in new_dict:
    for i in range(len(new_dict[key])):
        if new_dict[key][i] == None:
            continue
        elif isinstance(new_dict[key][i], list):
            lists_keys.append(key)
            break
        elif isinstance(new_dict[key][i], dict):
            dicts_keys.append(key)
            break
lists_keys.append('uuid')
lists_keys.append('number')
flat_lst = {}
flat_dict = {}
for key in lists_keys:
    flat_lst[key] = []
    for i in range(len(new_dict[key])):
        if new_dict[key][i] == None:
            flat_lst[key].append(None)
            continue
        flat_lst[key].append(new_dict[key][i])
    new_dict.pop(key)
# preciso pegar de novo e salvar, mas preciso do uuid para fazer o join depois
for key in dicts_keys:
    flat_dict[key] = []
    for i in range(len(new_dict[key])):
        # flat_dict[key][i]['uuid'] = new_dict['uuid'][i]
        if new_dict[key][i] == None:
            flat_dict[key].append(None)
            continue
        flat_dict[key].append(new_dict[key][i])
    new_dict.pop(key)
            

In [17]:
print(flat_lst.keys())
print(flat_dict.keys())
print(new_dict.keys())

dict_keys(['boosterTypes', 'uuid', 'finishes', 'frameEffects', 'keywords', 'promoTypes', 'subtypes', 'supertypes', 'types', 'variations'])
dict_keys(['identifiers', 'leadershipSkills', 'purchaseUrls'])
dict_keys(['asciiName', 'flavorText', 'hasFoil', 'hasNonFoil', 'isAlternative', 'isFullArt', 'isPromo', 'name', 'number', 'rarity', 'text', 'type', 'uuid'])


In [30]:
print(pformat(flat_dict['identifiers'][0]))
print(flat_dict.keys())

{'mcmId': '701764',
 'mcmMetaId': '417594',
 'mtgjsonV4Id': 'c2532311-a3bd-5ded-ae7a-169dc969cf70',
 'mtgoId': '113219',
 'scryfallId': '93de9042-cc62-4ade-8d8d-68fdbc84bfae',
 'scryfallIllustrationId': 'a6795e84-6dd5-4795-a481-bc26b0b13ca4',
 'scryfallOracleId': '3aa83ed2-f48b-4ce6-a614-2c54ddf50538'}
dict_keys(['identifiers', 'leadershipSkills', 'purchaseUrls'])


In [19]:
# create a dataframe from the dictionary
df = pd.DataFrame(new_dict)
# set uuid as primary key and number as index
df.set_index('uuid', inplace=True)
df.sort_values(by='number', inplace=True)
# save df to lotr.db
conn = sqlite3.connect('lotr.db')
df.to_sql('cards', conn, if_exists='replace', index=False)
conn.close()

In [33]:
for key in flat_lst.keys():
    print(key, len(flat_lst[key]))

boosterTypes 462
finishes 462
frameEffects 462
keywords 462
promoTypes 462
subtypes 462
supertypes 462
types 462
variations 462
uuid 462
number 462


In [35]:
df = pd.DataFrame(flat_lst)
df.set_index('uuid', inplace=True)
df.sort_values(by='number', inplace=True)
conn = sqlite3.connect('lotr.db')
df.to_sql('lists', conn, if_exists='replace', index=False)
conn.close()

ProgrammingError: Error binding parameter 2: type 'list' is not supported